# 태스크 5: 코드 생성을 위한 Bedrock 모델 호출

이 노트북에서는 대규모 언어 모델(LLM)을 사용하여 텍스트 프롬프트를 기반으로 코드를 생성하는 방법을 알아봅니다.

이 노트북에서 사용되는 프롬프트는 프롬프트 자체 외에는 텍스트의 예시를 제공하지 않으므로 제로샷 프롬프트입니다.

Amazon Bedrock에서 모델의 생성 기능을 시연하는 코드 생성 사용 사례를 살펴보겠습니다. Boto3 클라이언트를 사용하여 Amazon Bedrock API와 통신하는 방법을 살펴보고 내부 모델에 태스크, 지침 및 입력으로 구성된 입력을 API에 제공하여 추가 예시를 제공하지 않고 출력을 생성합니다. 여기에서 강력한 LLM에서 태스크를 쉽게 이해하고 설득력 있는 출력을 생성하는 방식을 입증하는 것이 목적입니다.


## 시나리오

당신은 AnyCompany의 데이터 분석가인 Moe입니다. 회사에서는 다양한 제품의 지난해 판매 실적을 이해하고 싶습니다. sales.csv라는 데이터세트를 제공받았습니다. 이 데이터 집합에는 다음 열이 포함되어 있습니다.

- 날짜(YYYY-MM-DD) 형식
- Product_ID(각 제품의 고유한 식별자)
- 가격(각 제품이 판매된 가격)


이 노트북에서는 주어진 프롬프트에 대한 코드를 생성하는 방법을 알아봅니다. Boto3 클라이언트에서 Amazon Bedrock API를 사용하는 Meta LLama 3를 사용합니다.

## 태스크 5.1: 환경 설정

이 태스크에서는 환경을 설정합니다.


In [ ]:
#create a service client by name using the default session.
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))


## 태스크 5.2: 코드 생성

이 태스크에서는 사용 사례에 대한 Python 프로그램을 생성하는 Amazon Bedrock 서비스에 대한 입력을 준비합니다.

#### 실습 설정 - 이 실습을 위한 샘플 sales.csv 데이터를 생성합니다.



In [ ]:
# create sales.csv file
import csv

data = [
    ["date", "product_id", "price", "units_sold"],
    ["2023-01-01", "P001", 50, 20],
    ["2023-01-02", "P002", 60, 15],
    ["2023-01-03", "P001", 50, 18],
    ["2023-01-04", "P003", 70, 30],
    ["2023-01-05", "P001", 50, 25],
    ["2023-01-06", "P002", 60, 22],
    ["2023-01-07", "P003", 70, 24],
    ["2023-01-08", "P001", 50, 28],
    ["2023-01-09", "P002", 60, 17],
    ["2023-01-10", "P003", 70, 29],
    ["2023-02-11", "P001", 50, 23],
    ["2023-02-12", "P002", 60, 19],
    ["2023-02-13", "P001", 50, 21],
    ["2023-02-14", "P003", 70, 31],
    ["2023-03-15", "P001", 50, 26],
    ["2023-03-16", "P002", 60, 20],
    ["2023-03-17", "P003", 70, 33],
    ["2023-04-18", "P001", 50, 27],
    ["2023-04-19", "P002", 60, 18],
    ["2023-04-20", "P003", 70, 32],
    ["2023-04-21", "P001", 50, 22],
    ["2023-04-22", "P002", 60, 16],
    ["2023-04-23", "P003", 70, 34],
    ["2023-05-24", "P001", 50, 24],
    ["2023-05-25", "P002", 60, 21]
]

# Write data to sales.csv
with open('sales.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(data)

print("sales.csv has been created!")

## 태스크 5.3: Amazon Bedrock에서 생성된 Python 프로그램으로 판매 분석

In [ ]:
# define prompt template
from langchain_core.prompts import PromptTemplate

def format_prompt(actor:str, input:str):
    match actor:
        case "user":
            prompt_template =  """<|begin_of_text|><|start_header_id|>{actor}<|end_header_id|>\n\n{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n
"""
            prompt = PromptTemplate.from_template(prompt_template)
            return prompt.format(actor=actor,input=input)
        case _:
            print("requested actor >" + actor + "< is not supported")
            return ""    

In [ ]:
# Create the prompt
# Analyzing sales

prompt_data = """

You have a CSV, sales.csv, with columns:
- date (YYYY-MM-DD)
- product_id
- price
- units_sold

Create a python program to analyze the sales data from a CSV file. The program should be able to read the data, and determine below:

- Total revenue for the year
- Total revenue by product 
- The product with the highest revenue 
- The date with the highest revenue and the revenue achieved on that date
- Visualize monthly sales using a bar chart

Ensure the code is syntactically correct, bug-free, optimized, not span multiple lines unnessarily, and prefer to use standard libraries. Return only python code without any surrounding text, explanation or context.

"""
prompt=format_prompt("user",prompt_data)

In [ ]:
body = json.dumps({
    "prompt": prompt,
    "max_gen_len": 2048,
    "temperature": 0,
    "top_p": 1,
    }) 
    

## 태스크 5.4: 모델 호출

In [ ]:
modelId = "meta.llama3-8b-instruct-v1:0"
response = bedrock_client.invoke_model(body=body, modelId=modelId)
response_body = json.loads(response.get('body').read())
output_list = response_body.get("generation", [])
print(output_list)

##### <i aria-hidden="true" class="far fa-copy"></i> (선택 사항) 인쇄된 출력에서 생성된 코드를 복사하고 Bedrock에서 생성된 아래 셀의 코드를 검증을 위해 실행합니다.


지금까지 Amazon Bedrock API에 실질적인 노출을 제공하는 `boto3` SDK를 사용하여 실험해 보았습니다. 이 API를 사용하여 지정된 매출 데이터를 분석하고 시각화하는 python 프로그램을 생성합니다.



### 직접 해보기

- 프롬프트를 특정 사용 사례로 변경하고 다양한 모델의 출력을 평가하십시오.
- 토큰 길이를 조작하면서 서비스의 지연 시간 및 응답성을 이해해 보십시오.
- 다양한 프롬프트 엔지니어링 원칙을 적용하여 더 나은 출력을 얻을 수 있습니다.

### 정리

이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 이 노트북 파일을 닫고 **태스크 6**을 계속합니다.